In [1]:
## Simple GenAI App using lanchain
import os
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
LANCHAIN_API_KEY = os.environ.get("LANCHAIN_API_KEY")
LANGCHAIN_PROJECT = os.environ.get("LANGCHAIN_PROJECT_NAME")

In [5]:
# Data Ingestion - From webside scrape the data

from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://python.langchain.com/docs/versions/v0_3/")
docs = loader.load()

In [ ]:
# Load Data -> Docs -> Divide text into chunks -> text to vector  -> Vector embedding -> Vector storage

In [6]:
# Divide text into chunks

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

In [7]:
# text to vectors
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [1]:
# Vector storage - FAISS
from langchain_community.vectorstores import FAISS

vector_stores_db = FAISS.from_documents(documents,embeddings)

#similarity search. Query from vector store DB
query = "Langsmith has to usage limits: total traces and extended"
result = vector_stores_db.similarity_search(query)
result[0].page_content

In [ ]:
# Retrieval chain, Document chain
from langchain_openai import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o")

prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based only on the provided context:
    <context>
    {context}
    </context>
    """
)

document_chain = create_stuff_documents_chain(llm,prompt)
document_chain

In [ ]:
from langchain_core.documents import Document

document_chain.invoke({
    "input" : "Langsmith has to usage limits: total traces and extended",
    "context" : [Document(page_content="Langsmith has to usage limits: total traces and extended")]
})

In [ ]:
## Input -> Retriever -> Vector store db

# As Retriever
retriever = vector_stores_db.as_retriever()

from langchain.chains import create_retrieval_chain
retriever_chain = create_retrieval_chain(retriever,document_chain)

# Get the response from the LLM
response = retriever_chain.invoke({
    "input" : "Langsmith has to usage limits: total traces and extended"
})
response['answer']

In [ ]:
# print the response
response